In [ ]:
import kagglehub
import shutil
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Download latest version
path = kagglehub.dataset_download("jaidenroman/team-4-rockpaperscissors")


print("Path to dataset files:", path)

100%|██████████| 216M/216M [00:06<00:00, 37.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jaidenroman/team-4-rockpaperscissors/versions/1


In [ ]:
# Define local save path
local_path = "./RPS_DS/"

# Ensure the local directory exists
os.makedirs(local_path, exist_ok=True)
# Copy the dataset to the local directory
for item in os.listdir(path):
    s = os.path.join(path, item)
    d = os.path.join(local_path, item)
    if os.path.isdir(s):
        shutil.copytree(s, d)
    else:
        shutil.copy2(s, d)

print(f"Dataset saved at: {local_path}")


Dataset saved at: ./RPS_DS/


In [ ]:
# Correcting paths
train_dir = "/content/RPS_DS/RPS_DS_2/TRAIN"
test_dir = "/content/RPS_DS/RPS_DS_2/TEST"

IMG_SZ = (300, 200)
BATCH_SZ = 32

# Define ImageDataGenerator for augmentation and rescaling
gen_train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

gen_test_data = ImageDataGenerator(rescale=1./255)

# Load training dataset
train_data = gen_train_data.flow_from_directory(
    train_dir,
    target_size=(IMG_SZ[0], IMG_SZ[1]),
    batch_size=BATCH_SZ,
    class_mode='categorical'
)

# Load testing dataset
test_data = gen_test_data.flow_from_directory(
    test_dir,
    target_size=(IMG_SZ[0], IMG_SZ[1]),
    batch_size=BATCH_SZ,
    class_mode='categorical'
)

Found 2016 images belonging to 3 classes.
Found 240 images belonging to 3 classes.


In [ ]:
#Automatically creates list of class names rather than defining manually
class_map = train_data.class_indices
class_names = list(class_map.keys())

In [ ]:
#Path where the trained model will be saved or loaded from.
model_path = "rockpaperscissors_model.h5"

#Keegan: Removed the original class_names list

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SZ[0], IMG_SZ[1], 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_data,
    epochs=30,
    validation_data=test_data
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 419s 7s/step - accuracy: 0.3454 - loss: 9.1277 - val_accuracy: 0.6458 - val_loss: 0.9666
Epoch 2/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 413s 7s/step - accuracy: 0.3882 - loss: 1.1945 - val_accuracy: 0.3917 - val_loss: 1.0694
Epoch 3/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 419s 7s/step - accuracy: 0.3936 - loss: 1.1363 - val_accuracy: 0.3833 - val_loss: 1.1322
Epoch 4/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 426s 7s/step - accuracy: 0.4090 - loss: 1.1273 - val_accuracy: 0.6208 - val_loss: 0.8940
Epoch 5/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 426s 7s/step - accuracy: 0.4458 - loss: 1.0528 - val_accuracy: 0.3375 - val_loss: 1.0924
Epoch 6/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 427s 7s/step - accuracy: 0.4547 - loss: 1.0464 - val_accuracy: 0.4125 - val_loss: 1.0593
Epoch 7/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 412s 7s/step - accuracy: 0.4544 - loss: 1.0381 - val_accuracy: 0.5833 - val_loss: 0.9399
Epoch 8/45
63/63 ━━━━━━━━━━━━━━━━━━━━ 447s 7s/step - accuracy: 0.5181 - loss: 0.9770 - val_accuracy: 0.5750 - v

In [ ]:
test_loss, test_acc = model.evaluate(train_data)
print(f"Test Accuracy: {test_acc:.2f}")
model.save(model_path)

In [ ]:
# === Predict from new image ===
def predict_new_image(img_path):
    model = load_model(model_path)

    img = Image.open(img_path).convert("RGB")
    img = img.resize(IMG_SZ)

    img_array = np.array(img).astype("float32") / 255.0
    img_array = img_array.reshape(1, IMG_SZ[1], IMG_SZ[0], 3)

    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    confidence = prediction[0][predicted_index]

    # Display
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Prediction: {class_names[predicted_index]} ({confidence:.2%})")
    plt.show()

In [ ]:
predict_new_image("/content/RPS_DS/RPS_DS_2/TEST/Paper/paper_712.png")


In [ ]:
predict_new_image("/content/RPS_DS/RPS_DS_2/TEST/Paper/qs7Zi0icFErU0fsZ.png")

In [ ]:
predict_new_image("/content/RPS_DS/RPS_DS_2/TEST/Scissor/20250609_112043.png")

In [ ]:
predict_new_image("/content/RPS_DS/RPS_DS_2/TEST/Paper/20250609_112823.png")

In [ ]:
predict_new_image("/content/RPS_DS/RPS_DS_2/TEST/Rock/stone_343.png")

In [ ]:
# === Visualization of training ===
print("Visualizing training results...")

# Plot accuracy
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
